In [1]:
#raspa tela
import requests
from bs4 import BeautifulSoup
#dataframe
import pandas as pd
#gráficos
import matplotlib.pyplot as plt
#data e hora.
import datetime as dtm
from datetime import date as dt
from unicodedata import normalize as nm
import time
import re
from dateutil import parser

# Informando período

In [2]:
#definir quais anos você quer analisar
AnoInicial=int(input("Informe o Ano inicial: "))
AnoFinal=int(input("Informe o Ano final: "))
print("Lembrando que quanto mais anos, mais demorado será a extração")

Informe o Ano inicial:  2023
Informe o Ano final:  2023


Lembrando que quanto mais anos, mais demorado será a extração
Lembrando que quanto mais anos, mais demorado será a extração


# Processo de Web Scraping

In [3]:
tempo_inicio = time.time()
linkRaiz = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
listaLinks = []
ListaFinal = []
jogo=[]
Info = []
Rodada = []

for ano in range(AnoInicial, AnoFinal+1):
    for nj in range(1, 4):
        linkJogo = f"{linkRaiz}/{ano}/{nj}"
        listaLinks.append(linkJogo)
        valorweb = requests.get(linkJogo)
        soup = BeautifulSoup(valorweb.content, 'html.parser')

        
        title = soup.title.text
        jogos=soup.find(class_='color-white block text-1').text
        jogo.append(jogos)
        ListaFinal.append(title)
        localdthr_jogo = soup.find_all(class_='text-2 p-r-20')
        Texto = [element.get_text(strip=True) for element in localdthr_jogo]
        Info.append(Texto)
       
        
        
tempo_final = time.time()
tempo_decorrido = tempo_final - tempo_inicio

print(ListaFinal)

print(f'Tempo decorrido: {tempo_decorrido:.2f} segundos')

['Flamengo - RJ 3 x 0 Coritiba S.a.f. - PR - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol', 'Botafogo - RJ 2 x 1 São Paulo - SP - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol', 'Palmeiras - SP 2 x 1 Cuiabá Saf - MT - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol']
Tempo decorrido: 1.52 segundos
['Flamengo - RJ 3 x 0 Coritiba S.a.f. - PR - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol', 'Botafogo - RJ 2 x 1 São Paulo - SP - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol', 'Palmeiras - SP 2 x 1 Cuiabá Saf - MT - Campeonato Brasileiro de Futebol - Série A 2023 - Confederação Brasileira de Futebol']
Tempo decorrido: 1.52 segundos


# Iniciar o Processo de DataFrame para acompanhamento dos Jogos

In [4]:
Local = []
Data = []
Hora = []
Cidade = []
Estadio = []
Estado = []
Dia = []
for Texto in Info:
    local = Texto[0]
    Local.append(local)
    data = Texto[1]
    hora = Texto[2]
    Hora.append(hora)
    dia = data.split(',')
    x = dia[0]
    Dia.append(x)
    i = dia[1]
    Data.append(i)

for i in Local:
    x = i.split('-')
    y = x[0]
    z = x[1]
    w = x[2]
    Estadio.append(y)
    Cidade.append(z)
    Estado.append(w)

In [6]:
#Coletar Número Jogo
NumeroJogo=[]
for i in jogo:
    j=i.split('Jogo:')[1].strip()
    NumeroJogo.append(j)

#Coletar Time da Casa
time_casa = []
UF_Time_Mandante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeCasa = times[0].strip()  # Remover espaços extras no início e no final
    nome_time_casa = timeCasa.split(' - ')[0].strip()  # Separar o nome do time da parte do estado
    Estado= timeCasa.split(' - ')[1].strip()
    Estado = Estado.split(' ')[0]
    time_casa.append(nome_time_casa )
    UF_Time_Mandante.append(Estado)


#coletar Visitante
time_visitante = []
UF_Time_Visitante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeVisitante = times[1].strip()  # Remover espaços extras no início e no final
    nome_time_visitante = timeVisitante.split(' - ')[0].strip() 
    nome_time_visitante1 = nome_time_visitante[2:]  # Remover o número no início
    time_visitante.append(nome_time_visitante1)
    Estado = timeVisitante.split('-')[1].strip()
    UF_Time_Visitante.append(Estado)



#coletar gols mandante
gols_mandante = []

for gols in ListaFinal:
    gol_mandante = gols.split(' x ')[0].strip()  # Extrair os gols do time da casa
    time = gol_mandante.split(' - ')[1].strip()
    time=time[3:]
    gols_mandante.append(time)

#Coletar gols visitante
gols_visitante=[]
for gols in ListaFinal:
    gols_visitantes=gols.split(' x ')[1].strip()
    gols_visitantes=gols_visitantes.split(' - ')[0].strip()
    gols_visitantes=gols_visitantes.split(' ')[0]
    gols_visitante.append(gols_visitantes)

#Coletar Ano
anoJogo=[]
for i in ListaFinal:
    Ano_Jogos=i.split(' x ')[1].strip()
    Ano_Jogos=Ano_Jogos.split(' - ')[3]
    Ano_Jogos=Ano_Jogos[-4:]
    anoJogo.append(Ano_Jogos)

In [7]:
#Determinar quem venceu
def vitoria(gols_mandante, gols_visitante):
    if gols_mandante > gols_visitante:
        return 'Mandante'
    elif gols_mandante < gols_visitante:
        return 'Visitante'
    else:
        return 'Empate'
resultado=[vitoria(gols_mandante,gols_visitante) for gols_mandante,gols_visitante in zip(gols_mandante,gols_visitante)]

In [8]:
#Criando o DataFrame 
data = {'NumeroJogo':NumeroJogo, 'Estadio':Estadio, 'Cidade':Cidade,'UF':Estado,'Hora':Hora, 'Data':Data, 'Quem Venceu?':resultado, 'AnoJogo':anoJogo, 'Time Mandante':time_casa,'UF Mandante': UF_Time_Mandante , 'Gols Mandante':gols_mandante,'Time Visitante':time_visitante,'UF Visitante': UF_Time_Visitante,'Gols Visitante':gols_visitante}
df = pd.DataFrame(data)


In [9]:
#Excluindo jogos que ainda não ocorreram 
df['Gols Visitante'] = pd.to_numeric(df['Gols Visitante'], errors='coerce').astype('Int64')
df['Gols Mandante'] = pd.to_numeric(df['Gols Mandante'], errors='coerce').astype('Int64')
df['NumeroJogo'] = pd.to_numeric(df['NumeroJogo'], errors='coerce').astype('Int64')
df['Rodada'] = df['NumeroJogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
df['Time Mandante'] = df['Time Mandante'].str.replace('.', '')
df['Time Visitante'] = df['Time Visitante'].str.replace('.', '')
df = df.dropna(subset=['Gols Visitante'])
X = ['NumeroJogo', 'Estadio', 'Cidade', 'UF', 'Hora', 'Data', 'Quem Venceu?', 'AnoJogo', 'Rodada', 'Time Mandante', 'UF Mandante', 'Gols Mandante', 'Time Visitante', 'UF Visitante', 'Gols Visitante']
df = df[X]
df

,NumeroJogo,Estadio,Cidade,UF,Hora,Data,Quem Venceu?,AnoJogo,Rodada,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,Mandante,2023,1,Flamengo,RJ,3,Coritiba Saf,PR,0
1,2,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,Mandante,2023,1,Botafogo,RJ,2,São Paulo,SP,1
2,3,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,Mandante,2023,1,Palmeiras,SP,2,Cuiabá Saf,MT,1


,NumeroJogo,Estadio,Cidade,UF,Hora,Data,Quem Venceu?,AnoJogo,Rodada,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,Mandante,2023,1,Flamengo,RJ,3,Coritiba Saf,PR,0
1,2,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,Mandante,2023,1,Botafogo,RJ,2,São Paulo,SP,1
2,3,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,Mandante,2023,1,Palmeiras,SP,2,Cuiabá Saf,MT,1


In [10]:
#Exportando Para Excel
df.to_excel(f"CampeonatoBrasileiro_Jogos_CBF_{AnoInicial}-{AnoFinal}_SerieA.xlsx", index = False)

### Acompanhar desempenho dos times Individualmente

In [11]:
#Copiar Dataframe anterior
dx_local = df[['NumeroJogo', 'Time Mandante','UF Mandante', 'Gols Mandante', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']].copy()
dx_visitante = df[['NumeroJogo', 'Time Visitante','UF Visitante', 'Gols Visitante', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data','AnoJogo']].copy()
dx_local.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_local['Mandante/Visitante'] = 'Mandante'
dx_visitante.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_visitante['Mandante/Visitante'] = 'Visitante'
#concatenar
dx = pd.concat([dx_local, dx_visitante], ignore_index=True)
#Demonstrar pontuação
dx['Pontos'] = dx.apply(lambda row: 3 if row['Quem Venceu?'] == row['Mandante/Visitante'] else (1 if row['Quem Venceu?'] == 'Empate' else 0), axis=1)
dx = dx.drop('Quem Venceu?', axis=1)
dx

,NumeroJogo,Time,UF Time,Gols,Estadio,Cidade,UF,Hora,Data,AnoJogo,Mandante/Visitante,Pontos
0,1,Flamengo,RJ,3,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,2023,Mandante,3
1,2,Botafogo,RJ,2,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,2023,Mandante,3
2,3,Palmeiras,SP,2,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,2023,Mandante,3
3,1,Coritiba Saf,PR,0,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,2023,Visitante,0
4,2,São Paulo,SP,1,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,2023,Visitante,0
5,3,Cuiabá Saf,MT,1,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,2023,Visitante,0


,NumeroJogo,Time,UF Time,Gols,Estadio,Cidade,UF,Hora,Data,AnoJogo,Mandante/Visitante,Pontos
0,1,Flamengo,RJ,3,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,2023,Mandante,3
1,2,Botafogo,RJ,2,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,2023,Mandante,3
2,3,Palmeiras,SP,2,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,2023,Mandante,3
3,1,Coritiba Saf,PR,0,Maracanã,Rio de Janeiro,MT,16:00,16 de Abril de 2023,2023,Visitante,0
4,2,São Paulo,SP,1,Nilton Santos,Rio de Janeiro,MT,18:30,15 de Abril de 2023,2023,Visitante,0
5,3,Cuiabá Saf,MT,1,Allianz Parque,Sao Paulo,MT,16:00,15 de Abril de 2023,2023,Visitante,0


In [118]:
for ano in range(AnoInicial, AnoFinal+1):
    for nj in range(301, 302):
        linkJogo = f"{linkRaiz}/{ano}/{nj}"
        listaLinks.append(linkJogo)
        valorweb = requests.get(linkJogo)
        soup = BeautifulSoup(valorweb.content, 'html.parser')
        

        jogos=soup.find(class_='color-white block text-1').text
        jogos = jogos.split(':')[1].strip()
        

        GolJog = soup.find_all(class_='hidden-sm hidden-md hidden-lg m-t-20')
        i = [element.get_text(strip=True) for element in GolJog]
        captTimes = soup.find_all(class_='time-nome color-white')
        Texto = [element.get_text(strip=True) for element in captTimes]
            
print(i)
print(Texto)
    

["Pedro 20' (1ºT)", "Nonato 32' (1ºT)Joaquim 44' (2ºT)"]
['Flamengo - RJ', 'Santos - SP']
["Pedro 20' (1ºT)", "Nonato 32' (1ºT)Joaquim 44' (2ºT)"]
['Flamengo - RJ', 'Santos - SP']


### Abrindo o arquivo e Iniciando novas consultas a partir do ano e a data informada